**11.1-1<br>Suppose that a dynamic set S is represented by a direct-address table T of length m. Describe a procedure that finds the maximum element of S. What is the worst-case performance of your procedure?**
1. We create a direct-address table as in *11.1_Direct_address_tables.ipynb*
2. Starting from first non-empty index in T, we keep track of the highest index, which was set initially to $-\infty$
3. The worst-case performance is O(n), where the largest index is located at the end of the table


In [ ]:
import numpy as np
class item:
    def __init__(self,key,data=None):
        self.key=key # key is necessary for each item
        self.data=data # satellite data to store info, not necessary 
        
class DA_tables:
    def __init__(self,m):
        self.array=[None]*m
        self.length=m
    def insert(self,x):
        self.array[x.key]=x
   
    def find_max(self):
       
        max_=-np.inf
        for i in range(self.length):
          
            if self.array[i] is not None and self.array[i].data>max_:
                max_=self.array[i].data
                
        return max_
"""create a direct address table T with m available slots"""
t=DA_tables(m=10)
"""for key [0,1,2,3], we insert numbers [9,4,5,-3.3]"""
list1=[9,4,5,-3.3] #data
for key,data in enumerate(list1):
    t.insert(item(key,data))
t.find_max()    


**11.1-2<br>A *bit-vector* is simply an array of bits (0s and 1s). A bit vector of length m takes much less space than an array of m pointers. Describe how to use a bit vector to represent a dynamic set of distinct elements with no satellite data. Dictionary operations should run in O(1) time.**

Becuase the elements carry no satellite data, we can simply define an empty slot as `0`, an occupied slot as `1`. 
* INSERT(x) defined `x` as `1`
* DELETE(x) defined `x` as `0`
* SEARCH(x) checks if `x==1`(occupied) or `x==0`(empty)

They all take O(1), no loops.

In [ ]:
class DA_tables_bitvector:
    def __init__(self,m):
        self.array=[0]*m
    def insert(self,x): #given that x in range(m)
        self.array[x]=1
    def delete(self,x): 
        self.array[x]=0
    def search(self,x):
        
        if self.array[x]==1:
            return x
        else:
            print ('the slot is empty')
            return
b=DA_tables_bitvector(m=10)
list1=[2,3,4,8] #data
for key in list1:
    b.insert(key)
b.search(2) 
b.search(0)

**11.1-3<br>Suggest how to implement a direct-address table in which the keys of store elements do not need to be distinct and the elements can have satellite data. All three dictionary operations (INSERT, DELETE and SEARCH) should run in O(1) time. (Don't forget that DELETE takes as an argument a pointer to an object to be deleted, not a key)**

We can define each slot `T(k)` in the table such that:
* It is a pointer to a **doubly linked list** `L` (recall session *12.2_Linked_lists.ipynb*) containing all the objects with the same key
    * We only need to modify `class Node` slightly by adding another attribute `data` that points to satellite data
    * If the slot is empty, `L.head is None``
* Special attention to **DELETE**:
    * To fulfil O(1) running time, we can only assign node in the list `L` to be deleted, such as `L.head` or `L.head.next`. 
    * `delete_head(k)` shows an example of deleting the head of a list in a slot `T(k)`: it follows *first-in, last-out* policy!

In [ ]:
class Node:
    def __init__(self,key,data):
        self.key=key
        self.data=data
        self.next=None #pointer attribute next
        self.prev=None #pointer attribute prev
        return

class DoublyLinkedList:
    def __init__(self):# create an empty list without node
        
        self.head=None #initial head=None
        return
    def list_insert(self,x):

        """ verify if the object x has a node-like structure 
        with attributes key, data next and prev.
        If not, build it by by calling class item"""
        if not isinstance(x,Node):
            new_node=Node(x)
        else:
            new_node=x

        """1.insert node before head.
        2. assign pointer next and prev btw head and new_node
        3. set new_node as head"""
        new_node.next=self.head
        if self.head is not None:
            self.head.prev=new_node

        self.head=new_node
        new_node.prev=None
        return
    def list_search(self,key):

        current_node=self.head

        while current_node is not None and current_node.key!=key:
            current_node=current_node.next

        return current_node, current_node.key
    
    def list_delete(self,node):
        """ update the pointer next"""
        if node.prev is not None: # if node is not at the head
            node.prev.next=node.next
            
        else:
            self.head=node.next #if node is at the head
        """ update the pointer prev"""
        if node.next is not None:
            node.next.prev=node.next
            
        return
    
class DA_table_nondistinct:
    def __init__(self,m):
        self.array=[None]*m
       
    def insert(self,x):
        """if the slot is empty, make it into a doubly linked list
        else, insert a new node"""
        if self.array[x.key] is None: 
            self.array[x.key]=DoublyLinkedList()
        self.array[x.key].list_insert(x)
       
    def search(self,k):
       
        if self.array[k].head is None: #if the array is empty
            print ('item not found in table')
        else:
            return self.array[k], self.array[k].head.key, self.array[k].head.data
    def delete_head(self, k): 
        
        if self.array[k].head is None:
            print ('item not found in table')
        else:
            self.array[k].list_delete(self.array[k].head)
            return 
        
"""create a direct address table T with m available slots"""
t=DA_table_nondistinct(m=10)
"""for key [0,1,2,3], we insert numbers [9,4,5,-3.3]"""
list1=[9,4,5,-3.3] #data
for key,data in enumerate(list1):
    t.insert(Node(key,data))
t.insert(Node(2,'repeated'))
t.insert(Node(2,'still repeated'))
t.delete_head(0)
t.search(0)    

**11.1-4 $\star$<br>We wish to implement a dictionary by using direct addressing on a huge array. At the start, the array entries may contain garbage, and initializing the entire array is impractical because of its size. Describe a scheme for implementing a direct address dictionary on a huge array. Each stored object should use O(1) space; the operations SEARCH, INSERT, and DELETE should take O(1) time each; and initializing the data structure should take O(1) Time. (Hint: Use an additional array, treated somewhat like a stack whose size is the number of keys actually stored in the dictionary, to help determine whether a given entry in the huge array is valid or not.)**

*will be added later if time permits*